### HMDA Data Preprocessing
This file covers initial data loading and cleaning for the HMDA dataset. 

The primary dataset for this project is residential mortgage application data released by the FFIEC per the Home Mortgage Disclosure Act, hereafter referred to as the HDMA dataset (see reference 2). Data is available from 2018 to 2022 which will serve as the time period for our analysis.

This dataset covers mortgage application for single family properties, multifamily properties and a smaller percentage of other real estate. A variety of supporting information is provided for each mortgage such as the action taken on that application, e.g., approved, denied, etc., FIPS codes for the property (state and county), mortgage type, loan amount and appraised property values, loan to value ratios, debt ratios, etc., as well as general social demographic details on the borrower. 

The data for our analysis covers 2018 to 2022 and the file for each year is quite large (~1GB), which is making consolidating the data quite challenging given local computing resource constraints. To circumvent this issue, the datafile for each year is being cleaned individually (see details below) and only the aggregate details need for each county are consolidated instead of consolidating at the application level. 

#### Details on data cleaning:
* This project will focus on the risk of flooding in areas expected to experience rising sea levels and hence will focus on the coastal counties of the United States (see reference [1]). As a first step in our data preparation, all non coastal properties  are removed from the dataset (Step1).

* The HMDA dataset contains single family homes, mult family homes as well as some rural properties. For this project, we are primarily interested in the property values for single family homes; multi family homes and other types of real estate are excluded (Step2).

* Our main variable of interest is the property_value. therefore any records with unusable values for this feature are excluded

* Since the HDMA dataset is at the mortgage application level (as opposed to the issued mortage level), it contains data on denied applications or applications that were approved but where the loans were not issued for some reason. We exclude all applications where there may be concerns about data completeness (see data dictionary for relevant codes)


In [13]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

In [14]:
def coastal_counties(path = ""):
    """
    Description: function to load the file of coastal counties. Note for this analysis,  only contiguous US territories 
    are considered. So Hawaii will be dropped
    I/P: Optional file path. See [1] in references for original source
    O/P:Series of coastal counties
    """
    cc = pd.read_excel(path + "Coastal/coastline-counties-list.xlsx", usecols = [0, 3, 4, 5], 
                       skiprows = 3, nrows = 255, dtype = str)
    cc.columns = ["state_county_FIPS", "county_name", "state_name", "ocean"]
    cc = cc[cc.state_name != "Hawaii"] 
#     cc.drop(labels = ["state"], axis = 1, inplace = True)
    
    #dropping 'county' from county names so we can match fema data
    cc["county_name"] = [name[:-7] if " County" in name else name for name in list(cc["county_name"])]

    return cc

ccounties = coastal_counties('data/')
ccounties.sample(3)

,state_county_FIPS,county_name,state_name,ocean
198,45053,Jasper,South Carolina,Atlantic
155,36005,Bronx,New York,Atlantic
114,23023,Sagadahoc,Maine,Atlantic


In [3]:
def hmda_datacleaning(df, ccounties):
    
    """ 
    Description: this function cleans the HMDA dataset to keep only the data needed for further analysis.
    I/P: original dataframe
    O/P: cleaned dataframe
    """
    #Step 1: COASTAL PROPERTIES 
    #to reduce the data size, all non-coastal counties are removed per reference [1] and only  Note for this analysis,
    #only contiguous US territories are considered; Hawaii, Puerto Rico, US Virgin Islands and other such territories
    #are not included in the analysis. Additionally, counties with great lakes coasts are not considerd since the focus
    #is on sea level rising. 
    df = df[df.county_code.isin(ccounties.state_county_FIPS)]
    
    
    
    #Step 2: SINGLE FAMILY HOMES
    #Keeping only single family homes and removing multi-family homes and rural properties (loan_type = 4; see reference 3):
    #<why are we removing multi-families
    df = df[df.loan_type != 4]
    df = df[df.derived_dwelling_category.apply(lambda x: 'Single Family' in x)]

    
    
    #Step 3: NON NUMERIC/MISSING PROPERTY VALUE
    #Removing any records with non numeric property values since those do not provide useful info
    df["property_value"] = pd.to_numeric(df["property_value"], errors='coerce')
    df.dropna(subset = "property_value", inplace = True)
    
   

    #Step 3: 
    #(A)we exclude those cases where the application was withdrawn by the applicant or where the file was
    #closed for incompleteness as we cannot be certain that the data for this record are complete and accurate
    df = df[~df.action_taken.isin([4,5])]
    
    
    #(B) For the applications that were denied, we exclude them if the denial reason was incomplete documentation
    df = df[~(df.action_taken.isin([3,7])) | (
                                             (df.action_taken.isin([3,7])) & (
                                                                            (~df.denial_reason_1.isin([6,7])) &
                                                                            (~df.denial_reason_2.isin([6,7])) &
                                                                            (~df.denial_reason_3.isin([6,7])) &
                                                                            (~df.denial_reason_4.isin([6,7])))
                                             )]
    denial_reason_cols = ["denial_reason_1", "denial_reason_2","denial_reason_3","denial_reason_4"]
    df.drop(labels = denial_reason_cols, axis = 1, inplace = True)
    
   
#     #removed - we are no longer reading in these columns
#     #Step 4: similarly, analysis of race and other demographic details is beyong the scope of this project so we drop 
#     #those columns
#     demographic_cols = []
#     for col in df.columns:
#         if 'ethnicity' in col or 'race' in col or 'sex' in col or 'age' in col:
#             demographic_cols.append(col) #the number of cols in our dataset should drop a lot after this
#     df.drop(labels = demographic_cols, axis = 1, inplace = True)
    
    
    
#     #Step 5: since the focus of this project is conventional mortgages and associated home prices,
#     #we exclude reverse mortgages and other lines of credit  
#     df = df[(df["reverse_mortgage"] == 2) & (df["open_end_line_of_credit"] == 2)]
#     df.drop(labels = ["reverse_mortgage","open_end_line_of_credit"], axis = 1, inplace = True)
    
   
    
    return df

In [4]:
def hmda_datasetcreation(year, path = ""):
    """ 
    Description: this function first loads the dataset for each year and then calls the cleaning function to reduce
    this large dataset to only the needed dataset (coastal counties and other cleaning). Finally, this cleaned dataset
    for each year is concatenated to create the full dataset
    
    I/P: Optional file path. See [1] in references for original source
    O/P: final consolidatd dataset
    """
    #data loading
    df_raw = pd.read_csv(path + "HMDA/"+ str(year) + ".zip", compression = "zip", 
                             dtype={'county_code': object, "census_tract" : object, 'derived_msa_md': object},
                             usecols = ['activity_year', 'derived_msa_md', 'county_code','census_tract', 
                                        'loan_type','action_taken', 'property_value', 'derived_dwelling_category',
                                        'reverse_mortgage','open_end_line_of_credit',
                                        'denial_reason_1', 'denial_reason_2', 'denial_reason_3', 'denial_reason_4'])
    
    #data cleaning
    df = hmda_datacleaning(df_raw, ccounties)
    
    
#     #formats get messed up so using pickle instead
#     #writing file to disk as csv
#     filename = 'cleaned_' + str(year)
#     compression_options = dict(method='zip', archive_name=f'{filename}.csv')
#     df.to_csv(f'{filename}.zip', compression=compression_options)
    
    
    #writing file to disk as a pickle file to preserve formats
    filename = 'cleaned_' + str(year)
    compression_options = dict(method='zip', archive_name=f'HMDA/{filename}.pkl')
    df.to_pickle(f'data/HMDA/{filename}.zip', compression=compression_options)
 
    return

In [ ]:
years = np.arange(2018, 2023)
for year in years:
    hmda_datasetcreation(year,'data/')

In [26]:
def hmda_consolidation_aggregation(path = ""):
    """ 
    Description: Because the raw files were so large, directly consolidating the datasets resulted in memory issues.
    This function first calculates the median property values by state/county/censustract FIPS codes and then aggregates
    the data to reduce file size. The aggregated data is written to disk for reuse
    
    I/P: Optional file path
    O/P: final aggregated and consolidatd dataset 2019 to 2022
    """
    
    #placeholder for aggregated results
    results_full = pd.DataFrame() #columns = ['county_code', 'property_value', 'year'])  
    
    years = np.arange(2018, 2023) 
    for year in years:
        
        #data loading
        df = pd.read_pickle(path + "HMDA/" + "cleaned_" + str(year) + ".zip", compression = "zip")  #column types are being preserved

        #aggregation
        results = df.groupby(['activity_year','county_code','census_tract'], as_index = False).agg({"property_value":"median"})
        
        #consolidation        
        #results_full = results_full.append(results, ignore_index = True) #append decpriciated
        results_full = pd.concat([results_full, results], axis=0)
    
    #merging with county file for county names
    results_full = results_full.merge(ccounties, left_on = "county_code", right_on = "state_county_FIPS")
#     results_full["ID"] = results_full['county_code'] + "_" + results_full['census_tract']
    
    
    #for some census_tracts, we do not have data for all five years which would make time series based comparison
    #impossible. dropping these values
#     _ = results_full.groupby('ID').size()
    _ = results_full.groupby('census_tract').size()
    _ = _[_ == 5] #meaning we have 5years of data
    census_tracts_w_full_data = list(_.index)
    results_full = results_full[results_full.census_tract.isin(census_tracts_w_full_data)]
    
    
    #writing to disk for easy access
    compression_options = dict(method='zip', archive_name='results_full.pkl')
    results_full.to_pickle('data/results_full.zip', compression=compression_options)
    
    return results_full


In [ ]:
hmda_consolidation_aggregation('data/')

In [27]:
#df = hmda_consolidation_aggregation()

In [25]:
#df.head()

,activity_year,county_code,census_tract,property_value,state_county_FIPS,county_name,state_name,ocean
0,2018,01003,01003010100,175000.0,01003,Baldwin,Alabama,Gulf of Mexico
1,2018,01003,01003010200,145000.0,01003,Baldwin,Alabama,Gulf of Mexico
2,2018,01003,01003010300,225000.0,01003,Baldwin,Alabama,Gulf of Mexico
3,2018,01003,01003010400,185000.0,01003,Baldwin,Alabama,Gulf of Mexico
4,2018,01003,01003010500,145000.0,01003,Baldwin,Alabama,Gulf of Mexico
